# 2nd Eddy Cross Disciplinary Symposium
## An end to end Machine Learning project
#### Predicting the geoeffectiveness of a CME
  
Today's workflow:
1. Analyze data
2. Try algorithms
3. Evaluate results
4. See what else can be done

By the end of this meeting I hope you will understand:
1. What the steps of a Machine Learning project are
2. How to use tools to better analyze and visualize data
3. The principles behind some Machine Learning concepts

## The prerequisites
### What we'll be using today:

<table>
    <th colspan="2">Data Analysis</th>
    <th colspan="2">Most ML tasks</th>
    <th colspan="2">Neural Nets</th>
    <th colspan="2">Visualization</th>
    <tr>
        <td colspan="2"><img src="https://miro.medium.com/max/3200/1*9v51-jsfHtk6fgAIYLoiHQ.jpeg" alt="Pandas logo" width="250"/></td>
        <td colspan="2"><img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/01/scikit-learn-logo.png" alt="Sklearn logo" width="250"/></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/Tensorflow_logo.svg/115px-Tensorflow_logo.svg.png" alt="Tensorflow logo" height="250"/></td>
        <td><img src="https://miro.medium.com/max/1200/1*DKu_54iqz6C-p6ndo7rO3g.png" alt="Keras logo" width="250"/></td>
        <td colspan="2"><img src="https://rapids.ai/assets/images/Plotly_Dash_logo.png" alt="Plotly & Dash logos" width="250"/></td>
    </tr>
    </table>
   
I have linked everything at the end of this notebook. 

**Do not worry about the following lines**

In [ ]:
# Install necessary packages (inside this environment only)
!pip install dash
!pip install umap-learn
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [1]:
# Import the necessary libraries and functions
import pandas as pd
import numpy as np
import dash
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import tensorflow as tf

from pandas_profiling import ProfileReport
from umap import UMAP

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers.experimental.preprocessing import Normalization

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Activation
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# Configure plot logging
log_dir = "../logs" 

### Utilities
**Do not worry about this lines either.**  
These are some functions that will help us along the way with pretty printing and plotting, mostly. Feel free to try and understand the code, but do not worry if you don't. This is not what Machine Learning is about.

In [4]:
# Function that plots confusion matrix
def plot_confusion_matrix(cf_matrix):
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_names,group_counts)]
    labels = np.asarray(labels).reshape(2,2)
    plt.figure(figsize=(4, 4))
    sns.heatmap(cf_matrix, xticklabels=[0,1], yticklabels=[0,1], annot=labels, fmt='', cmap='Blues')
    plt.title("Confusion matrix")
    plt.ylabel('Actual class')
    plt.xlabel('Predicted class')
    plt.show()

    
# Function that plots the confusion matrix 
# and prints the following metrics: accuracy, recall (true positive rate), precision, true negative rate
def evaluate_performance(actual, predicted):
    # Create confusion matrix
    cm = confusion_matrix(actual, predicted)
    # Plot confusion matrix
    plot_confusion_matrix(cm)
    # Compute metrics
    # np.round(a, b) rounds a value to 2 decimals
    accuracy = np.round(accuracy_score(actual, predicted) * 100, 2)
    recall = np.round(recall_score(actual, predicted) * 100, 2)
    precision = np.round(precision_score(actual, predicted) * 100, 2)
    # Compute True Negative Rate = True Negative / (True Negative + False Positive)
    tnr =  np.round(cm[0][0]/(cm[0][0] + cm[0][1]) * 100, 2) 
    # Print metrics
    print("Accuracy: {}%".format(accuracy))
    print("Recall (True Positive Rate): {}%".format(recall))
    print("True Negative Rate: {}%".format(tnr))
    print("Precision: {}%".format(precision))  
    
    
# Function to get prediction label for binary classification
def get_binary_prediction_type(predicted, actual):
    if (predicted == 1 and actual == 1):
        return 'TP'
    if (predicted == 1 and actual == 0):
        return 'FP'
    if (predicted == 0 and actual == 0):
        return 'TN'
    if (predicted == 0 and actual == 1):
        return 'FN'
    return 'Unkonwn'


# Function to create a more comprehensive prediction data frame
def get_predictions_df(numeric_columns_used, umap_embedding, test_examples, test_labels, predicted_labels):
    # Concatenate all given column
    predictions_df = pd.concat([test_examples, test_labels], axis=1)
    predictions_df['predicted'] = predicted_labels
    # Label the predictions with their type
    predictions_df['prediction_type'] = predictions_df.apply(lambda row: get_binary_prediction_type(row['predicted'], 
                                                                          row['geoeffective']), 
                                                                          axis=1)
    # Add UMAP values to the dataframe
    predictions_df['umap_0'] = umap_embedding[:, 0]
    predictions_df['umap_1'] = umap_embedding[:, 1]
    return predictions_df
    

    
# Function to plot the more comprehensive prediction data umap embedded
def plot_comprehensive_predictions_umap(df):
    hover_columns = df.columns[~df.columns.isin(['umap_0', 'umap_1',
                                                'geoeffective', 'prediction_type',
                                                'predicted'])]
    fig = px.scatter(df, x='umap_0', y='umap_1', color='prediction_type', 
                     color_discrete_sequence=["blue", "yellow", "green", "red"],
                     hover_data=df[hover_columns])
    fig.show()     
    
    
# Function to summarize Grid Search results
def summarize_results(grid_result):
    print("Best score: %f, using %s\n" % (grid_result.best_score_, grid_result.best_params_))
    mean_f1 = grid_result.cv_results_['mean_test_score']
    params = grid_result.cv_results_['params']
    for mean_f1, param in zip(mean_f1, params):
        print("Mean score = %f with: %r" % (mean_f1, param))    

        
# Function to create a LogisticRegression pipeline
def create_log_reg_pipeline(selected_features):
    # NOTE: you can choose less features

    # Create a preprocessor
    preprocessor_standard_scaling = ColumnTransformer([
            ("scale", StandardScaler(), selected_features)],
            remainder="passthrough") # allow uscaled features; 'drop' <=> do not use those columns 

    # Create a pipeline
    return Pipeline([("preprocessor", preprocessor_standard_scaling), 
                    ("classifier", LogisticRegression(random_state=seed, 
                                                     class_weight='balanced'))])

## The task

### Predict whether a CME will be geoeffective

<img src="https://www.nasa.gov/sites/default/files/styles/full_width/public/thumbnails/image/mars_cme.gif?itok=fLhkCigJ" alt="CME-NASA" width="400"/>

#### Understanding the terms:  
  
[**CME (Coronal Mass Ejection)**](https://www.swpc.noaa.gov/phenomena/coronal-mass-ejections) = _Large expulsion of plasma and magnetic field from the Sun's corona_   

[**GMS (Geomagnetic Storm)**](https://www.swpc.noaa.gov/phenomena/geomagnetic-storms) = _Major disturbance of Earth's magnetosphere_ 

[**Dst (index)**](https://www.swpc.noaa.gov/content/space-weather-glossary#d) = _A measure of variation in the geomagnetic field_; For this workshop, we consider that **values <=-30 (nT) indicate geomagnetic storms**.

**Geoeffective** = Having the capacity to produce a GMS (i.e. **Dst <= - 30**)

#### Understanding the task:  
Having a number of solar parameters characterizing a **CME** (solar explosion), predict whether a **geomagnetic storm will occur** (Earth's magnetic field will be disturbed strongly enough that the **Dst** index value will be **<= -30**)  

## The data

In [5]:
# Read csv data using pandas 
# The data is read in a DataFrame 
cmes_csv_url = 'https://raw.githubusercontent.com/andreeapricopi/cmes_ml_workshop/main/cmes.csv'
df = pd.read_csv(cmes_csv_url)

In [ ]:
# Look at the data
df

##### Legend:  
**cpa** = Central Position Angle (degrees)  
**aw** = Angular Width (degrees)   
**lin_speed** = Linear Speed (km/s)    
**acceleration** (m/s^2)  
**mass** (gram)  
**kinetic_energy** (erg)  
**dst** = the Dst index value (nT)  

Find more information about these columns [here, from the LASCO catalog](https://cdaw.gsfc.nasa.gov/CME_list/catalog_description.htm)

## 1. Data exploration, analysis and preprocessing
### Because results are as good as the data you feed to the learners
#### And they usually only understand numbers 
We will look for:  
   - missing values  
   - unexpected values  
   - correlations  
   - distributions  

In [ ]:
# Get a summary regarding the data
# See number of missing values and data type by column
df.info()
# See mean, standard deviation, min, max values by column
df.describe()

### Data preprocessing
1. Data cleaning
2. Data labeling
3. Data splitting

### Data cleaning

In [8]:
# Remove columns with too many missing values
df.drop(columns = ['mass', 'kinetic_energy'], inplace=True) #inplace=True is the equivalent of df = df.drop(...)

In [9]:
# Remove rows with missing values
df.dropna(inplace=True) #inplace=True is the equivalent of df = df.dropna()

In [10]:
# Replace Dst character values with 0
df.dst = pd.to_numeric(df.dst.apply(lambda dst: 0 if dst == '-' else dst))

### Data labeling

We need to create a label column based on Dst value for our binary classification.  
Remember that Dst <= -30 means geoeffective, otherwise not geoeffective.  
We could use True/False values for a new column called 'geoeffective', but since we would eventually still need numerical data (and therefore, we would need to encode these labels as numbers), we will create a numerical label from the beginning:
- 0 means not geoeffective (Dst > -30)
- 1 means geoeffective (Dst <= -30)

In [11]:
# Create labels based on Dst value
threshold = -30
df['geoeffective'] = df.dst.apply(lambda dst: 0 if dst > threshold else 1)

### Data splitting
We need to **separate inputs** (cpa, aw, lin_speed, acceleration) from **outputs** (geoeffective label).  
We need to **put aside a subset of data for an objective test**.  
However, we also want to test our data after training and make improvements based on the results, so we will also have a test-like subset called **_dev_** (_development_ or also reffered to as a _validation_ set). 

In [12]:
# Split into X (inputs; samples) and y (label; target value)
X = df.copy()
y = X.pop('geoeffective')

In [13]:
# Drop the Dst value
X.drop(columns=['dst'], inplace=True)

In the following lines we will use some randomization. In order to ensure we all have the same results and that, even more importantly, you get the same results each time you run this notebook, we need to set a fixed value to feed as _random_state_. As long as you use the same value, the results will be the same.  
Feel free to see what happens if you remove the "random_state=seed" lines or if you change the seed value.

In [14]:
# Set a seed for reproductibility 
seed = 42

<img src="train_test_split.png" alt="Train-Test split" width="600">

In [15]:
# Split into train and test sets
X_traindev, X_test, y_traindev, y_test = train_test_split(X, 
                                                          y, 
                                                          test_size=0.2, 
                                                          random_state=seed, 
                                                          shuffle=True,
                                                          stratify=y)

<img src="train_dev_split.png" alt="Train-Dev split" width="400">

In [16]:
# Split into train and validation (dev) sets
X_train, X_dev, y_train, y_dev = train_test_split(X_traindev, 
                                                  y_traindev, 
                                                  test_size=0.2, 
                                                  random_state=seed, 
                                                  shuffle=True,
                                                  stratify=y_traindev)

### Data analysis & visualization
We will not look at the test data so as not to 

In [ ]:
# In order to have both inputs and outputs for the analysis, we will merge back again 
# X_traindev and y_traindev in a new, temporary variable
traindev_df = pd.concat((X_traindev, y_traindev), axis=1)
traindev_df

In [ ]:
# Get a comprehensive, auto-generated report
profile_report = ProfileReport(traindev_df)
profile_report

### Data Visualization. 2D plot

In [19]:
# Get UMAP embedding (simplified, 2D data representation)
# Initialize UMAP
umap = UMAP(random_state=seed) # the default number of dimensions (components) is 2
# Get the actual embeddings
traindev_umap = umap.fit_transform(X_traindev)
dev_umap = umap.fit_transform(X_dev)

In [20]:
# Function to plot the UMAP embedding
def plot_umap(umap_embedding, color_labels):
    fig = px.scatter(umap_embedding, x=0, y=1, color=color_labels)
    fig.show()

In [ ]:
# Plot the UMAP embeddings
plot_umap(traindev_umap, y_traindev.astype('str'))
plot_umap(dev_umap, y_dev.astype('str'))

### How does a machine learn?

The machine is trying to do is <span style="color:green">**discover a function**</span> that <span style="color:green">**best describes**</span> the **relationship between the input and the target** value.
  
<span style="color:green">**Discovering the function**</span>  actually means **discovering the** <span style="color:green">**parameters**</span> of that function.  
  
For example, for a logistic regression, the function is:  
> $ f(X) = W * X + b $, 

where W = [w0, w1, .., wn], X = [x0, x1, .., xn] - the features  and b is the bias.  
  
So <span style="color:green">**training**</span> a logistic regression model means **finding the values for W and b** that result in the prediction - f(X) - being as close to the real value as possible.  
  
**In other words**: the model finds out how much each feature weighs so that in the future, when being presented with a series of values, it knows how to compute a prediction as accurate as possible.  

<img src="learning.gif" alt="Gif with simplified learning demo" width="600">

### Logistic Regression

In [22]:
# Define the model
log_reg_model = LogisticRegression(random_state=seed) # use random seed for the same random weight initialization

In [ ]:
# Train the model
log_reg_model.fit(X_train, y_train)

In [24]:
# Predict dev set values using the model
dev_predictions = log_reg_model.predict(X_dev)

In [ ]:
# Evaluate the accuracy of the predictions
print('Accuracy: ', accuracy_score(y_dev, dev_predictions))

In [ ]:
# Visually compare the true values and the predicted ones
plot_umap(dev_umap, y_dev.astype('str'))
plot_umap(dev_umap, dev_predictions.astype('str'))

In [ ]:
# Evaluate the recall and precision of the predictions
print('Recall: ', recall_score(y_dev, dev_predictions)) # True Positive Rate
print('Precision: ', precision_score(y_dev, dev_predictions)) # 1 - False Discovery Rate

In [ ]:
# Add class balance
log_reg_balanced = LogisticRegression(class_weight='balanced', random_state=seed)

# Train and predict
log_reg_balanced.fit(X_train, y_train)
dev_predictions = log_reg_balanced.predict(X_dev)

# Evaluate performance
evaluate_performance(y_dev, dev_predictions)

In [ ]:
# Look at how many iterations were performed
print('#iterations = ', log_reg_balanced.n_iter_)

### Add feature scaling
We will use standard scaling (i.e. substract the mean and divide by standard deviation):  
> $ x = \dfrac {x - mean}{std dev} $

Pay attention to whether the results or the number of iterations it takes to get those results change.

In [29]:
# Add feature scaling
# Select the features we want to use
selected_features = X_train.columns

# Create a preprocessor
preprocessor_standard_scaling = ColumnTransformer([
        ("scale", StandardScaler(), selected_features)],
        remainder="drop")

# Create a pipeline
log_reg_scaling_pipeline = Pipeline([("preprocessor", preprocessor_standard_scaling), 
                                     ("classifier", LogisticRegression(random_state=seed, 
                                                                       class_weight='balanced'
                                                                       ))])

In [ ]:
# Train and predict
log_reg_scaling_pipeline.fit(X_train, y_train)
dev_predictions = log_reg_scaling_pipeline.predict(X_dev)

# Evaluate
evaluate_performance(y_dev, dev_predictions)

In [ ]:
# Look at how many iterations were performed
print('#iterations = ', log_reg_scaling_pipeline['classifier'].n_iter_)

In [31]:
# Create a comprehensive dataframe for better prediction analysis 
comprehensive_dev_df = get_predictions_df(selected_features, dev_umap, X_dev, y_dev, dev_predictions)

In [ ]:
# Plot the newly created dataframe
plot_comprehensive_predictions_umap(comprehensive_dev_df)

In [ ]:
# Look at what we've learned

# Look at the weights 
print('W = ', log_reg_scaling_pipeline['classifier'].coef_)

# Look at the bias
print('b = ', log_reg_scaling_pipeline['classifier'].intercept_)

### Feed Forward Artificial Neural Network

### The simplified way you should imagine a neural network
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Neural_network_example.svg/330px-Neural_network_example.svg.png" alt="Simplified ANN architecture" width="250">

 
### A more complex neural network (containing more hidden layers)
<img src="https://www.researchgate.net/profile/Chuan-Lin-3/publication/333567419/figure/fig4/AS:765686351671297@1559565260413/Construction-of-the-deep-neural-network-DNN-model.jpg" alt="ANN architecture" width="600">

### What's going on inside a neuron
<img src="https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1547672259/2_i1cdwq.png" alt="Neuron" width="600"> 

### Some activation functions
<img src="https://www.researchgate.net/profile/Junxi-Feng/publication/335845675/figure/fig3/AS:804124836765699@1568729709680/Commonly-used-activation-functions-a-Sigmoid-b-Tanh-c-ReLU-and-d-LReLU.ppm" alt="Activation functions" width="600">

In [34]:
# Take care of the class imbalance
# Compute the balanced weights
balanced_traindev_weights = compute_sample_weight('balanced', y_traindev)
balanced_train_weights = compute_sample_weight('balanced', y_train)
balanced_dev_weights = compute_sample_weight('balanced', y_dev)

In [35]:
# Take care of the data scaling
# Create the Normalization layer
normalizer = Normalization()
normalizer.adapt(np.array(X_train))

In [36]:
# Get the input shape of the model
input_shape = X_train.shape[1:] # the number on input neurons (i.e. number of columns)

# Function to create the model (build_fn)
def create_model(input_shape, learning_rate, neurons):  
    # Create a model that includes the normalization layer
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape)) # Specify the number of input neurons
    model.add(normalizer)
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Configure the optimizer
    optimizer = Adam(learning_rate=learning_rate)
    # Compile the model
    model.compile(loss='binary_crossentropy',
                 optimizer=optimizer)
    
    return model

In [37]:
# Set initial hyperparameters
neurons = 1
learning_rate = 1e-3 # How much do we change the weights after each step?
batch_size = 32 # How many samples do we take into consideration before making a change?
epochs = 50 # How many times do we go throught the entire data?

In [ ]:
##################################################################################################
# The following lines are for configuring where to store the tensorboard logs and how to name them
# Do not worry if you do not understand them, they have no influence over the machine learning part
# Create the name of the model
model_name = "n{}_lr{}_b{}_e{}_".format(neurons, learning_rate, batch_size, epochs)
# Tensorboard configurations
log_dir = "logs/fit/" + model_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
##################################################################################################

# Initialize the model
model = create_model(input_shape=input_shape, neurons=neurons, learning_rate=learning_rate)

# Fit the model on the actual trainining test
history = model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  sample_weight=balanced_train_weights,  
                  verbose=0,
                  validation_split=0.2,  
                  callbacks=[tensorboard_callback])

In [ ]:
# Display the learning curves
%tensorboard --logdir logs/fit

In [ ]:
# Predict using the trained model
dev_predictions = model.predict(X_dev)
# You don't need to be concerned about the following line, it is a simple implementation detail
# You can comment the line and see what happens without it if you're curious and try to understand the reason
# Hint: it has to do with the last layer in the network 
dev_predictions = dev_predictions.reshape(dev_predictions.shape[0]) 

# Visually compare the true values and the predicted ones
plot_umap(dev_umap, y_dev.astype('str'))
plot_umap(dev_umap, dev_predictions)

In [ ]:
# Evaluate the predictions
evaluate_performance(y_dev, dev_predictions >= 0.5)

### Searching for the optimal hyperparameters

Instead of going by hand through all the options, training the model using all the combinations, testing the trained model and then looking through all the results to find the best performance, we can use an out of the box scikit-learn implementation: **GridSearch** 

#### Intermezzo: K-Fold Cross-Validation

As hard as we try to select a representative value for the test set, there is no guarantee that we manage to do so and, therefore, the test results can be biased (either too optimistic or too pesimistic). Having more test sets could lead to a more accurate assessment. 
  
Therefore, you could:  
1. Split the dataset into k fold (subsets)  
2. Train on k - 1 folds
3. Test on the remaining fold  
4. Repeat steps 2 and 3 k times (until you've tested the model on all k folds)  
  
--> The performance of your model is the mean of all k test results
  
An example for K = 5:
![K-Fold Cross-Validation](https://zitaoshen.rbind.io/project/machine_learning/machine-learning-101-cross-vaildation/featured.png)

### Searching for optimal hyperparameters

### TODO
Following the example below, please fill in the gap with your own hyperparameters to try. Then execute the search and look at the results (by running the other 3 cells).  
  
> **Example:**  
> parameters_to_try = {  
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
'parameter_name_1': [1, 2, 3, 4],   
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
'parameter_name_1': ['a', 'b']   
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> }  
                    
**NOTE 1.**: The more parameters or values you add, the longer the search takes. For this workshop, please limit to a small number of possible combinations (eg: 3 parameters with 2 values each)  
  
**NOTE 2.**: Learning rate values are in the range (0, 1]  
Common values for batch sizes are 16, 32, 64, 128

In [1]:
# Define what options you want to explore
# Create a parameter grid
parameters_to_try = {
                    
                    }
# Hyperparameters examples: number of neurons (neurons), batch size (batch_size), 
# number of epochs (epochs), learning rate (learning_rate)

# !
From the cell below, **please delete the hyperparameters you want to tune**.  
Otherwise, they will take the default value and no search will actually be performed.

In [45]:
# Wrap the Keras model for GridSeeach (scikit-learn use):
# Specify the function that creates the model (build_fn), the batch size, the number of epochs, 
# the number of neurons and learning rate
# NOTE: KerasClassifier has a default threshold of 0.5, 
# meaning all values < 0.5 are considered 0s and all values >= 0.5 are considered 1s
model = KerasClassifier(build_fn=create_model, 
                        input_shape=input_shape,
                        ####################################################################################
                        ######################## TODO: DELETE HYPERPARAMETERS ##############################
                        # !!!
                        # Delete the lines that contain hyperparameters that you have defined in the grid
                        # aka in the cell above (parameters_to_try)
                        # as they will take those values.
                        # The ones you leave here will take the default values we've specified earlier (i.e. above)
                        neurons=neurons,
                        batch_size=batch_size,
                        epochs=epochs,
                        learning_rate=learning_rate,
                        # !!!
                        ###################################################################################
                        verbose=0)

# ?
What metric should we use to compare 2 models?

In [46]:
# GridSearch configuration
grid_search = GridSearchCV(estimator=model, 
                           param_grid=parameters_to_try, 
                           scoring=make_scorer(f1_score), # "make_scorer" is just an implementation detail
                           cv=3, 
                           verbose=2)

#### After executing the code in the 2 cells below, you should see something similar to this:  
  
<img src="grid_search_example.png" alt="Example of what you should see after GridSearch" width="800">

In [ ]:
# Perform GridSearch 
grid_search.fit(X_traindev, y_traindev, sample_weight=balanced_traindev_weights)

In [ ]:
# Look at the GridSearch results        
summarize_results(grid_search)        

### Is this actually ok? Look at the learning curves
Set your hyperparameters and execute the cells below to see the learning curves

In [ ]:
# Set YOUR hyperparameters here (the best ones you've found with GridSearch above)
neurons = 1
learning_rate = 0.001 
batch_size = 32 
epochs = 20 

In [ ]:
##################################################################################################
# The following lines are for configuring where to store the tensorboard logs and how to name them
# Do not worry if you do not understand them, they have no influence over the machine learning part
# Create the name of the model
model_name = "n{}_lr{}_b{}_e{}_".format(neurons, learning_rate, batch_size, epochs)
# Tensorboard configurations
log_dir = "logs/fit/" + model_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
##################################################################################################

# Initialize the model
model = create_model(input_shape=input_shape, neurons=neurons, learning_rate=learning_rate)

# Fit the model on the actual trainining test
history = model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  sample_weight=balanced_train_weights,  
                  verbose=0,
                  validation_split=0.2,  
                  callbacks=[tensorboard_callback])

In [ ]:
# Look at the learning curves
%tensorboard --logdir logs/fit

### Feature engineering

#### Polynomial features

In [53]:
# Add polynomial features
poly = PolynomialFeatures(include_bias=False)
X_traindev_poly = pd.DataFrame(poly.fit_transform(X_traindev))
X_train_poly = pd.DataFrame(poly.fit_transform(X_train))
X_dev_poly = pd.DataFrame(poly.fit_transform(X_dev))
X_test_poly = pd.DataFrame(poly.fit_transform(X_test))

#### Test a model with polynomial features, using cross validation 
We will use a scikit-learn cross validation implementation

In [54]:
# Implementation details

# Specify the new input_shape (i.e. number of input neurons)
input_shape=X_traindev_poly.shape[1:]

# Use an appropriate normalization layer
normalizer=Normalization()
normalizer.adapt(np.array(X_traindev_poly))

In [55]:
# Initialize the model (as a KerasClassifier since we're using scikitlearn )
model = KerasClassifier(build_fn=create_model,
                        neurons=1,
                        input_shape=input_shape,
                        learning_rate=0.005,
                        batch_size=32,
                        epochs=epochs,
                        verbose=0)

# Remember how we used to supply "sample_weight" as a parameter to the "fit" methos before?
# We need to do this again here, but since cross validation is a very general concept, 
# its implementation accepts many other parameters
# Therefore, define 'fit_params'
fit_parameters = {
    'sample_weight': balanced_traindev_weights
}
# We want to see all metrics:
metrics = {
    'accuracy': make_scorer(accuracy_score), 
    'recall': make_scorer(recall_score), 
    'precision': make_scorer(precision_score), 
    'f1': make_scorer(f1_score)
}    

In [ ]:
# Cross validate
cv_results = cross_validate(model, 
                         X_traindev_poly, 
                         y_traindev, 
                         fit_params=fit_parameters,
                         return_train_score=True,
                         scoring=metrics) 

# NOTE: we can also specify k, the number of folds, by 'cv' parameter
# By default, k = 5

# Look at the results for each fold
cv_results

In [ ]:
# Look at the means of the performance metrics
print('---TRAIN---')
print('Mean accuracy: ', np.mean(cv_results['train_accuracy'] * 100))
print('Mean recall: ', np.mean(cv_results['train_recall'] * 100))
print('Mean precision: ', np.mean(cv_results['train_precision'] * 100))
print('---TEST---')
print('Mean accuracy: ', np.mean(cv_results['test_accuracy'] * 100))
print('Mean recall: ', np.mean(cv_results['test_recall'] * 100))
print('Mean precision: ', np.mean(cv_results['test_precision'] * 100))

# ?
Are the values corresponding to training an testing similar?  
Is that good/bad?

#### Test the logistic regression with polynomial features, using cross validation 

In [ ]:
# Create model
model = create_log_reg_pipeline(X_traindev.columns)

# Cross valiidate
cv_results = cross_validate(model, X_traindev, y_traindev, scoring=metrics, return_train_score=True, cv=3)

In [ ]:
# Look at the results for each fold
cv_results

In [ ]:
# Look at the means of the performance metrics
print('---TRAIN---')
print('Mean accuracy: ', np.mean(cv_results['train_accuracy'] * 100))
print('Mean recall: ', np.mean(cv_results['train_recall'] * 100))
print('Mean precision: ', np.mean(cv_results['train_precision'] * 100))
print('---TEST---')
print('Mean accuracy: ', np.mean(cv_results['test_accuracy'] * 100))
print('Mean recall: ', np.mean(cv_results['test_recall'] * 100))
print('Mean precision: ', np.mean(cv_results['test_precision'] * 100))

### Final test

In [ ]:
# Choose the best model so far
# best_model = #TODO

In [ ]:
# Train the best model on the entire training set (ie. traindev)
#TODO

In [ ]:
# Look at the learning curves
#TODO

In [ ]:
# Make predictions (using X_test)
#TODO

In [ ]:
# Evaluate the model (compare to y_test)
#TODO

## Recap
1. A machine learning project has the following steps:
    1. Problem definition. Data gathering
    2. Data exploration and preprocessing
    3. **Repeated trial and improvement**
    4. Final testing
2. **Training** means iterating repeatedly over the data and altering the weights to decrease the prediction error.  
3. A **neural network** is a computational system that maps inputs to outputs. Creating it means finding the optimal hyperparameters. There is no one size fits all, experimenting is required.
4. **Data cleaning and preprocessing** (eg: scaling, normalization) is very important
5. **Data imbalance** is a serious issue and must be addressed for good results
6. The **metrics** we use are important (eg: some can be misleading)


##### What did we use today?
- [pandas](https://pandas.pydata.org/), together with [pandas-profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/)  
- [NumPy](https://numpy.org/)  
- [scikit-learn](https://scikit-learn.org/stable/)  
- [TensorFlow](https://www.tensorflow.org/overview)  
- [Keras](https://keras.io/)  
- [UMAP](https://umap-learn.readthedocs.io/en/latest/)  
- [Plotly](https://plotly.com/python/)
 
##### Want to know more?
Besides scikit-learn, Tensorflow and Keras' websites, which include nice tutorials, I recommend this book:  
[Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow, 2nd Edition, by Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) as well as this blog: [machinelearningmastery.com](https://machinelearningmastery.com/blog/)
    
  
    
**Contact**: andreeapricopi@gmail.com